# Getting Started

In this notebook, we will walk through the most basic functionalities of SmartSim.

 - Creating and Running Models
 - Creating and Running Ensembles
 - Running and Communicating with the Orchestrator
 - Ensembles using SmartRedis


## 1.1 Running Models 

`Experiment`s are how users define workflows in SmartSim. The `Experiment` is used to create `Model` instances which represent applications, scripts, or largely any program. An experiment can start and stop a `Model` and monitor execution.

We begin by importing the `Experiment` class.

In [1]:
from smartsim import Experiment

After having done so, our next step is to initialize an `Experiment` instance. To do this we must provide the `Experiment` a name. This can be any string, but it is best practice to give it a meaningful name as a broad title for what types of models the experiment will be supervising. For our purposes, we will name our `Experiment` `"getting-started"`.

We also will also tell the `Experiment` which launcher it should use to run models. We do this by setting the `launcher` argument in the constructor. Accepted values for this argument are `"slurm"`, `"pbs"`,
`"cobalt"`, `"lsf"`, `"local"`, or `"auto"`. If `launcher="auto"` is used, the experiment will attempt to find a launcher on the system, and use the first one it encounters. If a launcher cannot be found or no launcher parameter is provided, the default value of `launcher="local"` will be used. 

For simplicity, we will start on a single machine, and as such will set the launcher argument to `"local"`

In [2]:
# Init Experiment and specify to launch locally
exp = Experiment(name="getting-started", launcher="local")

Throughout this tutorial, we will incrementally build upon this `Experiment`. We start from the simplest case: Creating and running a single `Model` instance

Our first `Model` will simply print `hello`, using the shell command `echo`.

We use `Experiment.create_run_settings` to create a `RunSettings` instance for our `Model`. `RunSettings` help parameterize *how* a `Model` should be executed provided the system and available computational resources.

`create_run_settings` is a factory method that will instantiate a `RunSettings` object of the appropriate type based on the `run_command` argument (i.e. `mpirun`, `aprun`, `srun`, etc). The default argument of `auto` will attempt to choose a `run_command` based on the available system software and the launcher specified in the experiment. If `run_command=None` is provided, the command will be launched without one.

In [3]:
# settings to execute the command "echo hello!"
settings = exp.create_run_settings(exe="echo", exe_args="hello!", run_command=None)

# create the simple model instance so we can run it.
M1 = exp.create_model(name="tutorial-model", run_settings=settings)

Once the `Model` has been created by the `Experiment`, it can be started.

By setting `summary=True`, we can see a summary of the experiment printed before it is launched. The summary will stay for 10 seconds, and it is useful as a last check. If we set `summary=False`, then the experiment would be launched immediately.

We also explicitly set `block=True` (even though it is the default), so that  `Experiment.start` waits until the last `Model` has finished before returning: it will act like a job monitor, letting us know if processes run, complete, or fail.

In [4]:
exp.start(M1, block=True, summary=True)



=== LAUNCH SUMMARY ===
Experiment: getting-started
Experiment Path: /Users/mrdro/repos/ssimdev/SmartSim/tutorials/01_getting_started/getting-started
Launching with: local
# of Ensembles: 0
# of Models: 1
Database: no

=== MODELS ===
tutorial-model
Model Run Settings: 
Executable: /bin/echo
Executable arguments: ['hello!']







15:03:44 C02G13RYMD6N SmartSim[50584] INFO tutorial-model(52842): Completed


The model has completed. Let's look at the content of the current working directory. We can see that two files, `tutorial-model.out` and `tutorial-model.err` have been created.

In [5]:
outputfile = './tutorial-model.out'
errorfile = './tutorial-model.err'

print("Content of tutorial-model.out:")
with open(outputfile, 'r') as fin:
    print(fin.read())
print("Content of tutorial-model.err:")
with open(errorfile, 'r') as fin:
    print(fin.read())

Content of tutorial-model.out:
hello!

Content of tutorial-model.err:



The `.out` file contains the output generated by `tutorial-model`, and the `.err` file would contain the error messages generated by it. Since there were no errors, the `.err` file is empty.

Now let's run two different `Model` instances at the same time. This is just as easy as running one `Model`, and takes the same steps. This time, we will skip the summary. 

In [6]:
run_settings_1 = exp.create_run_settings(exe="echo", exe_args="hello!", run_command=None)
run_settings_2 = exp.create_run_settings(exe="sleep", exe_args="5", run_command=None)
model_1 = exp.create_model("tutorial-model-1", run_settings_1)
model_2 = exp.create_model("tutorial-model-2", run_settings_2)
exp.start(model_1, model_2)

15:03:49 C02G13RYMD6N SmartSim[50584] INFO tutorial-model-1(52853): Completed
15:03:52 C02G13RYMD6N SmartSim[50584] INFO tutorial-model-2(52854): Running
15:03:53 C02G13RYMD6N SmartSim[50584] INFO tutorial-model-2(52854): Completed


For users of parallel applications, launch binaries (run commands) can also be specified in `RunSettings`. For example, if `mpirun` is installed on the system, we can run a model through it, by specifying it as `run_command` in `create_run_settings`.

Please note that to run this you need to have OpenMPI installed. 

In [7]:
# settings to execute the command "mpirun -np 2 echo hello world!"
openmpi_settings = exp.create_run_settings(exe="echo",
                                           exe_args="hello world!",
                                           run_command="mpirun")
openmpi_settings.set_tasks(2)

# create and start the MPI model
ompi_model = exp.create_model("tutorial-model-mpirun", openmpi_settings)
exp.start(ompi_model, summary=True)



=== LAUNCH SUMMARY ===
Experiment: getting-started
Experiment Path: /Users/mrdro/repos/ssimdev/SmartSim/tutorials/01_getting_started/getting-started
Launching with: local
# of Ensembles: 0
# of Models: 1
Database: no

=== MODELS ===
tutorial-model-mpirun
Model Run Settings: 
Executable: /bin/echo
Executable arguments: ['hello', 'world!']
Run Command: mpirun
Run arguments: {'n': 2}






15:04:10 C02G13RYMD6N SmartSim[50584] INFO tutorial-model-mpirun(52878): Completed


This time, since we asked `mpirun` to run two tasks by calling `openmpi_settings.set_tasks(2)`, in the output file we should find the line `hello world!` twice.

In [8]:
outputfile = './tutorial-model-mpirun.out'

print("Content of tutorial-model-mpirun.out:")
with open(outputfile, 'r') as fin:
    print(fin.read())


Content of tutorial-model-mpirun.out:
hello world!
hello world!



## 1.2 Running Ensembles

In the previous example, the two `Model` instances were created separately. There are more convenient ways of doing this, through `Ensemble`s. `Ensemble`s are groups of `Model` instances that can be treated as a single reference. We start by specifying `RunSettings` similar to how we did with our `Model`s.

In [9]:
# define how we want each ensemble member to execute
# in this case we create settings to execute "sleep 3"
ens_settings = exp.create_run_settings(exe="sleep", exe_args="3")

Then, instead of creating a single model like we did in previously, we will call the `Experiment.create_ensemble` method to create an `Ensemble`. Let's assume we want to run the same experiment four times in parallel. We will then pass the method the same arguemnts that we might pass `Experiment.create_model` in addition to the `replicas=4` argument. Finally, we simply start the `Ensemble` the same way we wold start a `Model`.

In [10]:
ensemble = exp.create_ensemble("ensemble-replica",
                               replicas=4,
                               run_settings=ens_settings)

exp.start(ensemble, summary=True)



=== LAUNCH SUMMARY ===
Experiment: getting-started
Experiment Path: /Users/mrdro/repos/ssimdev/SmartSim/tutorials/01_getting_started/getting-started
Launching with: local
# of Ensembles: 1
# of Models: 0
Database: no

=== ENSEMBLES ===
ensemble-replica
# of models in ensemble: 4
Launching as batch: False
Run Settings: 
Executable: /bin/sleep
Executable arguments: ['3']







15:04:28 C02G13RYMD6N SmartSim[50584] INFO ensemble-replica_0(52897): Completed
15:04:28 C02G13RYMD6N SmartSim[50584] INFO ensemble-replica_2(52899): Completed
15:04:29 C02G13RYMD6N SmartSim[50584] INFO ensemble-replica_1(52898): Completed
15:04:29 C02G13RYMD6N SmartSim[50584] INFO ensemble-replica_3(52900): Completed
15:04:30 C02G13RYMD6N SmartSim[50584] INFO ensemble-replica_1(52898): Completed
15:04:30 C02G13RYMD6N SmartSim[50584] INFO ensemble-replica_3(52900): Completed


From the output, we see that four copies of our `Model`, named *ensemble-replica_0*, *ensemble-replica_1*, ... were run. In each output file, we will see that the same output was generated.

Now let's imagine that we don't want to run the *same* model four times, but we want to run variations of it. One way of doing this would be to define four models, and starting them through the `Experiment`.

For a few, simple `Model`s, this would be OK, but what if we needed to run a large number of models, which only differ for some parameter? Defining and adding each one separately would be tedious. For such cases, we will rely on a parameterized `Ensemble` of models.

Say we had a python file `output_my_parameter.py` with this contents:
```py
# contents of output_my_parameter.py
import time

time.sleep(2)
print("Hello, my name is ;tutorial_name; " + 
      "and my parameter is ;tutorial_parameter;")
```

Our goal is to run 

```python output_my_parameter.py```

with multiple parameter values substituted where the text contains `;tutorial_name;` and `;tutorial_parameter;`. Clearly, we could pass the parameters as arguments, but in some cases, this could not be possible (e.g. if the parameters were stored in a file or the executable would not accept them from the command line).

First thing first, is that we must again create our run settings:

In [11]:
rs = exp.create_run_settings(exe="python", exe_args="output_my_parameter.py")

Then, we define the parameters we are going to set and values for those parameters in a dictionary. In this example, we are setting:

 1. `tutorial_name` with values `"Ellie"` and `"John"`
 2. `tutorial_parameter` with values `2` and `11`
 
In the original file `output_my_parameter.py`, which acts as a template, they occur as `;tutorial_name;` and `;tutorial_parameter;`. The semi-colons are used to perform a regexp substitution with the desired values. The semi-colon in this case, is called a *tag* and can be changed.

We pass the parameter ditionary to `Experiment.create_ensemble`, along with the argument `perm_strategy="all_perm"`. This argument means that we want all possible permutations of the given parameters, which are stored in the argument `params`. We have two options for both parameters, thus our ensemble will run 4 instances of the same `Experiment`, just using a different copy of `output_my_parameter.py` created by calling `Experiment.generate()`. We attach the template file to the `Ensemble` instance, generate the augmented python files, and run the experiment.

In [12]:
params = {
    "tutorial_name": ["Ellie", "John"],
    "tutorial_parameter": [2, 11]
}
ensemble = exp.create_ensemble("ensemble", params=params, run_settings=rs, perm_strategy="all_perm")

# to_configure specifies that the files attached should be read and tags should be looked for
config_file = "./output_my_parameter.py"
ensemble.attach_generator_files(to_configure=config_file)

exp.generate(ensemble, overwrite=True)
exp.start(ensemble)

15:04:39 C02G13RYMD6N SmartSim[50584] INFO ensemble_0(52918): Completed
15:04:39 C02G13RYMD6N SmartSim[50584] INFO ensemble_1(52919): Completed
15:04:39 C02G13RYMD6N SmartSim[50584] INFO ensemble_2(52920): Completed
15:04:40 C02G13RYMD6N SmartSim[50584] INFO ensemble_3(52923): Completed
15:04:41 C02G13RYMD6N SmartSim[50584] INFO ensemble_3(52923): Completed


We can see from the output that four instances of our experiment were run, each one named like the `Experiment`, with a numeric suffix at the end: `ensemble_0`, `ensemble_1`, etc. The call to ``Experiment.generate()`` created isolated output directories for each created `Model` in the ensemble and each ensemble member generated its own output files, which was stored in its respective directory.

In [13]:
for id in range(4):
    outputfile = f"getting-started/ensemble/ensemble_{id}/ensemble_{id}.out"

    print(f"Content of {outputfile}:")
    with open(outputfile, 'r') as fin:
        print(fin.read())


Content of getting-started/ensemble/ensemble_0/ensemble_0.out:
Hello, my name is Ellie and my parameter is 2

Content of getting-started/ensemble/ensemble_1/ensemble_1.out:
Hello, my name is Ellie and my parameter is 11

Content of getting-started/ensemble/ensemble_2/ensemble_2.out:
Hello, my name is John and my parameter is 2

Content of getting-started/ensemble/ensemble_3/ensemble_3.out:
Hello, my name is John and my parameter is 11



That's it! All possible permutations of the input parameters were used to execute the experiment! Sometimes, the parameter space can be too large to be explored exhaustively. In that case, we can use a different permutation strategy, i.e. `random`. For example, if we want to only use two possible random combinations of our parameter space, we can run the following code, where we specify `n_models=2` and `perm_strategy="random"`.

In [14]:
params = {
    "tutorial_name": ["Ellie", "John"],
    "tutorial_parameter": [2, 11]
}
ensemble = exp.create_ensemble("ensemble", params=params, run_settings=rs, perm_strategy="random", n_models=2)
config_file = "./output_my_parameter.py"
ensemble.attach_generator_files(to_configure=config_file)

exp.generate(ensemble, overwrite=True)
exp.start(ensemble)

15:04:45 C02G13RYMD6N SmartSim[50584] INFO Working in previously created experiment
15:04:49 C02G13RYMD6N SmartSim[50584] INFO ensemble_0(52947): Completed
15:04:49 C02G13RYMD6N SmartSim[50584] INFO ensemble_1(52948): Completed


Another possible permutation strategy is `stepped`, but it is also possible to pass a function, which will need to generate combinations of parameters starting from the dictionary. Please refer to the documentation to learn more about this.


It is also possible to use different delimiters for the parameter regexp. For example, if we had simmlarly parameterized file named `output_my_parameter_new_tag.py`, with contents:
```py
# Contents of output_my_parameter_new_tag.py
import time

time.sleep(2)
print("Hello, my name is @tutorial_name@ " + 
      "and my parameter is @tutorial_parameter@")
```

We would want to use `@`, instead of `;`, as our *tag*. We can trivially make this adaptation by passing a `tag` argument to our `Experiment.generate` call.

In [15]:
rs = exp.create_run_settings(exe="python", exe_args="output_my_parameter_new_tag.py")
params = {
    "tutorial_name": ["Ellie", "John"],
    "tutorial_parameter": [2, 11]
}
ensemble = exp.create_ensemble("ensemble_new_tag",
                               params=params,
                               run_settings=rs,
                               perm_strategy="all_perm")

config_file = "./output_my_parameter_new_tag.py"
ensemble.attach_generator_files(to_configure=config_file)

exp.generate(ensemble, overwrite=True, tag='@')
exp.start(ensemble)

15:04:50 C02G13RYMD6N SmartSim[50584] INFO Working in previously created experiment
15:04:55 C02G13RYMD6N SmartSim[50584] INFO ensemble_new_tag_0(52959): Completed
15:04:55 C02G13RYMD6N SmartSim[50584] INFO ensemble_new_tag_1(52960): Completed
15:04:55 C02G13RYMD6N SmartSim[50584] INFO ensemble_new_tag_2(52961): Completed
15:04:56 C02G13RYMD6N SmartSim[50584] INFO ensemble_new_tag_3(52962): Completed
15:04:57 C02G13RYMD6N SmartSim[50584] INFO ensemble_new_tag_3(52962): Completed


Last, we can see all the kernels we have executed by calling `Experiment.summary()`

In [16]:
print(exp.summary())

|    | Name                  | Entity-Type   |   JobID |   RunID |    Time | Status    |   Returncode |
|----|-----------------------|---------------|---------|---------|---------|-----------|--------------|
|  0 | tutorial-model        | Model         |   52842 |       0 | 2.00355 | Completed |            0 |
|  1 | tutorial-model-1      | Model         |   52853 |       0 | 2.21388 | Completed |            0 |
|  2 | tutorial-model-2      | Model         |   52854 |       0 | 6.01196 | Completed |            0 |
|  3 | tutorial-model-mpirun | Model         |   52878 |       0 | 2.00386 | Completed |            0 |
|  4 | ensemble-replica_0    | Model         |   52897 |       0 | 4.63194 | Completed |            0 |
|  5 | ensemble-replica_2    | Model         |   52899 |       0 | 4.21996 | Completed |            0 |
|  6 | ensemble-replica_1    | Model         |   52898 |       0 | 6.42562 | Completed |            0 |
|  7 | ensemble-replica_3    | Model         |   52900 |       0

## 1.3 Running and Communicating with the Orchestrator

In this section we will see how to use `SmartRedis` clients to interact with an in-memory database launched by SmartSim called the `Orchestrator`. We start by importing the SmartRedis `Client` and the `Orchestrator` from SmartSim

In [17]:
from smartredis import Client
from smartsim.database import Orchestrator
import numpy as np

REDIS_PORT=6899

We start the `Orchestrator`. Since we are setting `launcher="local"` in `Experiment`, the `Orchestrator` will run a single DB instance.

In [18]:
exp = Experiment("tutorial-smartredis", launcher="local")

# create and start a database
orc = Orchestrator(port=REDIS_PORT)
exp.generate(orc)
exp.start(orc, block=False)

Now that the `Orchestrator` is running, we can use SmartRedis to store NumPy tensors on the Redis DB, and get them back. This is done using the SmartSim `Client`. First, we setup a connection to the DB.

In [19]:
client = Client(address=f"127.0.0.1:{REDIS_PORT}", cluster=False)

Then, we can use the DB to put and retrieve tensors. We need to assign a unique key to each tensor (or object) we store on the DB.

In [20]:
send_tensor = np.ones((4,3,3))

client.put_tensor("tutorial_tensor_1", send_tensor)

receive_tensor = client.get_tensor("tutorial_tensor_1")

print('Receive tensor:\n\n', receive_tensor)

Receive tensor:

 [[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]


With the SmartRedis `Client` and its possible to store and run a Pytorch neural network directly on the DB node. We first create a one-layer PyTorch Convolutional Neural Network, and save it as a jit-traced, serialized, object.

In [21]:
import torch
import torch.nn as nn

# taken from https://pytorch.org/docs/master/generated/torch.jit.trace.html
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 1, 3)

    def forward(self, x):
        return self.conv(x)


net = Net()
example_forward_input = torch.rand(1, 1, 3, 3)
module = torch.jit.trace(net, example_forward_input)

# Save the traced model to a file
torch.jit.save(module, "./torch_cnn.pt")   

Now we send the model to the database, again, we assign it a unique key, `tutorial-cnn`, which we will use to refer to the model when using the `Client`.

In [22]:
# Set the model in the Redis database from the file
client.set_model_from_file("tutorial-cnn", "./torch_cnn.pt", "TORCH", "CPU")

Now we create a random tensor, store it on the DB, and use it as input to the CNN we just sent. The `Orchestrator` will run the neural network and store the output with the key we specify. Using that key, we can retrieve the tensor.

In [23]:
# Put a tensor in the database as a test input
data = torch.rand(1, 1, 3, 3).numpy()
client.put_tensor("torch_cnn_input", data)

# Run model and retrieve the output
client.run_model("tutorial-cnn", inputs=["torch_cnn_input"], outputs=["torch_cnn_output"])
out_data = client.get_tensor("torch_cnn_output")

Notice that we could have defined the model as an object (without storing it on disk) and send it to the DB using `set_model` instead of `set_model_from_file`. We can do the same thing for any Python function. For example, let's define a simple function takes a NumPy tensor as input.

In [24]:
def max_of_tensor(array):
    """Sample torchscript script that returns the
    highest element in an array.

    """
    # return the highest element
    return array.max(1)[0]

sample_array_1 = np.array([np.arange(9.)])
print(sample_array_1)
print("Max:")
print(max_of_tensor(sample_array_1))

[[0. 1. 2. 3. 4. 5. 6. 7. 8.]]
Max:
8.0


Now let's store this function on the DB, assigning it the key `max-of-tensor`: 

In [25]:
client.set_function("max-of-tensor", max_of_tensor)

Now we perform the same sample computation on the DB. 

In [26]:
client.put_tensor("script-data-1", sample_array_1)
client.run_script(
    "max-of-tensor",  # key of our script
    "max_of_tensor",  # function to be called
    ["script-data-1"],
    ["script-output"],
)

out = client.get_tensor("script-output")

print(out)

[8.]


And, as expected, we obtain the same result we obtained when we ran the function locally. To clean up, we need to tear down the DB. We do this by stopping the `Orchestrator`.

In [27]:
exp.stop(orc)

15:05:32 C02G13RYMD6N SmartSim[50584] INFO Stopping model orchestrator_0 with job name orchestrator_0-CH9X5FLRWKY0


## 1.4 Ensembles using SmartRedis

In Section 1.2 we used `Ensemble`s. What would happen if `Model`s which are part of an `Ensemble` tried to put their tensors on the DB using SmartRedis? Unless we used unique keys across the running programs, several tensors (or objects) would have the same key, and this key collision would result in unexpected behavior. In other words, if in the source code of one program, a tensor with key `tensor1` was put on the DB, then each replica of the program would put a tensor with the key `tensor1`. SmartSim and SmartRedis can avoid key collision by prepending program-unique prefixes to entities. 

Let's start by setting up the experiment with the `Orchestrator`.

In [28]:
exp = Experiment("tutorial-smartredis-ensemble", launcher="local")

# create and start a database
orc = Orchestrator(port=REDIS_PORT)
exp.generate(orc)
exp.start(orc, block=False)

Now let's add two replicas of the same `Model`. Basically, it is a simple producer, which puts a tensor on the DB. The code for it is in `producer.py`.

In [29]:
rs_prod = exp.create_run_settings("python", f"producer.py --redis-port {REDIS_PORT}")
ensemble = exp.create_ensemble(name="producer",
                               replicas=2, 
                               run_settings=rs_prod)

We add a consumer, which will just retrieve the tensors put by the two producers and check that they are what it expects.

In [30]:
rs_consumer = exp.create_run_settings("python", f"consumer.py --redis-port {REDIS_PORT}")
consumer = exp.create_model("consumer", run_settings=rs_consumer)

We need to register incoming entities, i.e. entities for which the prefix will have to be known by other entities. When we will start the `Experiment`, environment variables will be set to let all entities know which incoming entities are present.

In [31]:
consumer.register_incoming_entity(ensemble.models[0])
consumer.register_incoming_entity(ensemble.models[1])

Finally, we attach the files to the experiments, generate them, and run!

In [32]:
ensemble.attach_generator_files(to_copy=['producer.py'])
consumer.attach_generator_files(to_copy=['consumer.py'])
exp.generate(ensemble, overwrite=True)
exp.generate(consumer, overwrite=True)

# start the models
exp.start(ensemble, consumer, summary=True)

15:06:03 C02G13RYMD6N SmartSim[50584] INFO Working in previously created experiment
15:06:03 C02G13RYMD6N SmartSim[50584] INFO Working in previously created experiment


=== LAUNCH SUMMARY ===
Experiment: tutorial-smartredis-ensemble
Experiment Path: /Users/mrdro/repos/ssimdev/SmartSim/tutorials/01_getting_started/tutorial-smartredis-ensemble
Launching with: local
# of Ensembles: 1
# of Models: 1
Database: no

=== ENSEMBLES ===
producer
# of models in ensemble: 2
Launching as batch: False
Run Settings: 
Executable: /Users/mrdro/miniconda3/envs/ssim/bin/python
Executable arguments: ['producer.py', '--redis-port', '6899']



=== MODELS ===
consumer
Model Run Settings: 
Executable: /Users/mrdro/miniconda3/envs/ssim/bin/python
Executable arguments: ['consumer.py', '--redis-port', '6899']







15:06:17 C02G13RYMD6N SmartSim[50584] INFO producer_0(53055): Completed
15:06:17 C02G13RYMD6N SmartSim[50584] INFO consumer(53057): Completed
15:06:19 C02G13RYMD6N SmartSim[50584] INFO producer_1(53056): Completed


The producers produced random NumPy tensors, and we can see that the consumer was able to retrieve both of them from the DB, by looking at its output.

In [33]:
outputfile = './tutorial-smartredis-ensemble/consumer/consumer.out'

with open(outputfile, 'r') as fin:
    print(fin.read())

Tensor for producer_0 is: [[[[0.19524054 0.1577657  0.26017183]
   [0.57358988 0.22707672 0.02206434]
   [0.88094173 0.44048731 0.11940281]]]]
Tensor for producer_1 is: [[[[0.85576333 0.25515467 0.14493229]
   [0.09237036 0.55807815 0.79487725]
   [0.91174237 0.88470207 0.07428644]]]]



As usual, let's shutdown the DB, by stopping the `Orchestrator`.

In [34]:
exp.stop(orc)

15:06:19 C02G13RYMD6N SmartSim[50584] INFO Stopping model orchestrator_0 with job name orchestrator_0-CH9X5TYK5Y34
